In [ ]:
# Importing required packages

from math import ceil
import math
import numpy as np
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import glob
import re
from collections import OrderedDict
import os
import json

from numpy.lib.function_base import average
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

import matplotlib.lines as mlines
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
calendar.setfirstweekday(6)

In [ ]:
# Add your Data Files Path here
data_files_path = '/content/gdrive/MyDrive/Data Files/Release Data'

In [ ]:
# Loading the dataset path into variables
Location = 'India-1'
harmonics_data = data_files_path + f'/Harmonics data/{Location}/{Location}_harmonics.csv'
power_data = data_files_path + f'/Power consumption data/{Location}/{Location}.csv'

In [ ]:
# Defining a Custom Pallete, that we will use in the later parts for representing the Colors of the States Identified
sns.set(rc={'figure.figsize':(20,8)})

palette = {
    '0': list(sns.color_palette())[0],
    '1': list(sns.color_palette())[1],
    '2': list(sns.color_palette())[2],
    '3': list(sns.color_palette())[3],
    '4': list(sns.color_palette())[4],
    '5': list(sns.color_palette())[5],
    '6': list(sns.color_palette())[6],
    '7': list(sns.color_palette())[7],
}


In [ ]:
# Function to preprocess the dataset CSV files. Function return a Dataframe object with datetime set as Index
# Please change the Time Zone convertor based on the dataset being used (US/Eastern or Asia/Kolkata)
def preprocessing_data( path ):

    file_path = Path(path)
    print(file_path)
    df = pd.read_csv(file_path)

    df = df.rename(columns={'Unnamed: 0': 'datetime'})
    df['datetime'] = pd.to_datetime(df['datetime']) 
    df = df.set_index('datetime')
    # df = df.tz_convert('US/Eastern')
    df = df.tz_convert('Asia/Kolkata')
    df = df.resample('1min').mean()

    return df

In [ ]:
# Function to obtain the Pricipal Components from the PCA algorithm
# The function return a Dataframe with datetime as Index and multiple principle components as Columns of the dataframe
def pca_function(df , components):

  if 'ActivePT' in df.columns:
    feature_array = df.drop( ['ActivePT'], axis=1 ).values
  else:
    feature_array = df.values

  pca = PCA(n_components=components)
  principalComponents = pca.fit_transform(feature_array)
  principalDf = pd.DataFrame(data = principalComponents, columns = ['components_'+str(i+1) for i in range(components) ] )
  principalDf.index = df.index

  return principalDf

In [ ]:
# preprocessing the power and harmonics data CSV files and storing them as DataFrames
df_power = preprocessing_data(power_data)
df_harmonics = preprocessing_data(harmonics_data)

## Data Preprocessing and Interpolation
In the following lines of code, we will preprocess the data.

1. We choose only the necessary features from the Power and Harmonics DataFrames for usage in the following steps.
2. We try to check how much of the datapoints are missing.
3. We use interpolating functions to fill in the missing data.

In [ ]:
# Filtering the required features from Power and Harmonics DataFrames
df_power = df_power[['ActivePT']]
df_harmonics = df_harmonics[[ "AI_HR3", "AI_HR5", "AI_HR7", "AI_HR9", 'AI_HR11', 'AI_HR13', 'AI_HR15', 'AI_HR17', 'AI_HR19', 'AI_HR21', 'AI_HR23', 'AI_HR25', 'AI_HR27', 'AI_HR29', 'AI_HR31',
                              "BI_HR3", "BI_HR5", "BI_HR7", "BI_HR9", 'BI_HR11', 'BI_HR13', 'BI_HR15', 'BI_HR17', 'BI_HR19', 'BI_HR21', 'BI_HR23', 'BI_HR25', 'BI_HR27', 'BI_HR29', 'BI_HR31', 
                              "CI_HR3", "CI_HR5", "CI_HR7", "CI_HR9", 'CI_HR11', 'CI_HR13', 'CI_HR15', 'CI_HR17', 'CI_HR19', 'CI_HR21', 'CI_HR23', 'CI_HR25', 'CI_HR27', 'CI_HR29', 'CI_HR31',
                            ]]

In [ ]:
# Merging the both the power and harmonics DataFrames into one df_merged
# This cells also prints the percentage of the missing data.
df_merged = pd.merge( df_harmonics, df_power, left_index=True, right_index=True)
print(f'Missing data: {round(df_merged.isna().sum().sum()/(len(df_merged)*len(df_merged.columns)),4)*100}%')

In [ ]:
#Functions to replace nan values - Functions for Interpolation

def search_past_dates( data, timestamp, col ):
  # print('Searching for values in past dates to ',timestamp)
  start_timestamp = data.index[0]
  end_timestamp = timestamp - timedelta(days=1)

  time_value = timestamp.strftime('%H:%M:%S')

  start_date = start_timestamp.date()
  end_date = end_timestamp.date()

  if start_date > end_date:
    return False

  period = (end_date - start_date).days + 1

  for dt in pd.date_range( start_date, periods = period )[::-1]:
    timestamp_str = str(dt.date()) +' ' + str(time_value)
    timestamp_var = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S' )

    if timestamp_str < start_timestamp.strftime( '%Y-%m-%d %H:%M:%S' ):
      break

    if timestamp_var not in data.index:
      continue

    value = data.loc[timestamp_str][col]

    if math.isnan( value ) == False:
      return timestamp_str

  return False


def search_future_dates( data, timestamp, col ):
  # print('Searching for values in future dates to ',timestamp)
  start_timestamp = timestamp + timedelta(days=1)
  end_timestamp = data.index[-1]

  time_value = timestamp.strftime('%H:%M:%S')

  start_date = start_timestamp.date()
  end_date = end_timestamp.date()

  if start_date > end_date:
    return False

  period = (end_date - start_date).days + 1

  for dt in pd.date_range( start_date, periods = period ):
    timestamp_str = str(dt.date()) +' ' + str(time_value)
    timestamp_var = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S' )

    if timestamp_str > end_timestamp.strftime( '%Y-%m-%d %H:%M:%S' ):
      break

    if timestamp_var not in data.index:
      continue

    value = data.loc[timestamp_str][col]

    if math.isnan( value ) == False:
      return timestamp_str

  return False
    
# Fill the missing data in the dataframe - Interpolting the dataframe

def interpolate_dataframe(df):
  curr_month = 0
  for idx in df.index:

    if idx.date().month != curr_month:
      curr_month = idx.date().month
      # print(idx.date())

    if df.loc[idx].isnull().values.any() == True:

      if math.isnan( df.loc[idx]['ActivePT'] ) == False:
        past_idx = search_past_dates( df, idx, 'AI_HR3' )
        future_idx = search_future_dates( df, idx, 'AI_HR3' )

        # print(idx, past_idx, future_idx)

        if past_idx == False:
          df.loc[idx, df.columns!='ActivePT'] = df.loc[future_idx, df.columns!='ActivePT']
          continue

        if future_idx == False:
          df.loc[idx, df.columns!='ActivePT'] = df.loc[past_idx, df.columns!='ActivePT']
          continue

        df.loc[idx, df.columns!='ActivePT'] = (df.loc[past_idx, df.columns!='ActivePT'] + df.loc[future_idx, df.columns!='ActivePT'])/2

      if math.isnan( df.loc[idx]['ActivePT'] ) == True:
        past_idx = search_past_dates( df, idx, 'ActivePT' )
        future_idx = search_future_dates( df, idx, 'ActivePT' )

        # print(idx, past_idx, future_idx)

        if past_idx == False:
          df.loc[idx]['ActivePT'] = df.loc[future_idx]['ActivePT']
          continue

        if future_idx == False:
          df.loc[idx]['ActivePT'] = df.loc[past_idx]['ActivePT']
          continue

        df.loc[idx] = (df.loc[past_idx] + df.loc[future_idx])/2

  return df

In [ ]:
# Calling the funtion for interpolating the missing data and checking for percentage of missing data after interpolation
df_merged_filled = interpolate_dataframe(df_merged.copy())
print(f'Missing data: {round(df_merged_filled.isna().sum().sum()/(len(df_merged_filled)*len(df_merged_filled.columns)),4)*100}%')

In [ ]:
# The remaining NaN values (if any) are filled with the mean value
if df_merged_filled.isnull().values.any():
  df_merged_filled = df_merged_filled.fillna( df_merged_filled.mean() )
df_merged_filled.isnull().values.any(), round(df_merged_filled.isna().sum().sum()/(len(df_merged_filled)*len(df_merged_filled.columns)),4)*100

## Clustering and Classification

In the following lines of Code, let us look at different functions that were modeled to obtain the States from the above processed Data

1. We define different functions with the clustering algorithm and for training a classification model.
2. Further we will see the implementation of elbow method, which we can use to define the optimal number of clusters that can be used.

In [ ]:
# KMeans Clustering Algorithm implemented in a function.
# use 'return_centers = True' as an argument to get the centers of the clusters found.
# By default the algorithm returns the labels of cluster the datapoint belongs to in a list and the inertia value from the model.
def cluster_kmeans(data, n_clusters, return_centers=False):

  model = KMeans(n_clusters=n_clusters, random_state=100000).fit(data)
  centers = model.cluster_centers_
  X_labels = model.labels_

  if return_centers == True:
    return [str(i) for i in X_labels], centers
  else:
    return [str(i) for i in X_labels], model.inertia_

In [ ]:
# The following function return F1_Score and Silhouette Score, given the data, classification model and number of clusters.
def get_f1_score( data, model, num_clusters ):
  prediction_labels = model.predict(data)
  kmeans_labels, _ = cluster_kmeans(np.array(data), num_clusters)

  if len(set(kmeans_labels)) > 1:
    cluster_silScore  = metrics.silhouette_score(np.array(data), kmeans_labels, metric='euclidean')
  else:
    cluster_silScore = 0
    
  f1_score = metrics.f1_score(kmeans_labels, prediction_labels, average='micro' )

  return f1_score, cluster_silScore

In [ ]:
# Training the Classification model
# Takes a dataframe and number of clusters as arguments to train the Random Forest Classifier model.
# Function return a trained classification model

def train_cluster_classification_model(  data, number_of_clusters ):

  if 'ActivePT' in data.columns:
    flag = 1
  else:
    flag = 0

  labels = []
  centers = []
  if flag == 1:
    X = np.array(data.drop(['ActivePT'], axis=1))
  else:
    X = np.array(data)

  labels, centers_list = cluster_kmeans( X, number_of_clusters, return_centers=True )

  data['cluster_labels'] = labels

  if flag == 1:
    train_X = data.drop(['ActivePT','cluster_labels'], axis=1)
    train_y = data['cluster_labels']
  else:
    train_X = data.drop(['cluster_labels'], axis=1)
    train_y = data['cluster_labels']

  model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
  model.fit(train_X,train_y)

  return model


In [ ]:
# Storing the Dates in a DataFrame
dates_in_dataframe = set([])
for i in df_merged_filled.index:
  if i.date() not in dates_in_dataframe:
    dates_in_dataframe.add(i.date())
dates_in_dataframe = sorted(list(dates_in_dataframe))

In [ ]:
# Finding the start and end dates of the dataset
df_merged_filled.index[0], df_merged_filled.index[-1]

In [ ]:
# Defining the Training Start Date and Training End Date
train_start = f'{dates_in_dataframe[0].year}-{dates_in_dataframe[0].month}-{dates_in_dataframe[0].day}'
train_end = f'{dates_in_dataframe[6].year}-{dates_in_dataframe[6].month}-{dates_in_dataframe[6].day}'

In [ ]:
# The following code cell generates two plots
# 1. Number of Clusters vs Inertia of the model
# 2. Number of Clusters vs Silhouette Score
# These both plots are used to examine and determine the optimal number of clusters, 
# which can be used further for clustering and classification
fig,ax = plt.subplots(1,2, figsize=(10,4))

inertia_list = []
silhouette_list = []
for k in range(2,20):
  test_model = KMeans(n_clusters=k, random_state=100000).fit(df_merged_filled.loc[train_start:train_end].drop(['ActivePT'], axis=1))
  inertia_list.append(test_model.inertia_)
  silhouette_list.append(metrics.silhouette_score(np.array(df_merged_filled.loc[train_start:train_end].drop(['ActivePT'], axis=1)), test_model.labels_, metric='euclidean'))

sns.lineplot( x = list(range(2,20)), y = inertia_list, ax=ax[0])
ax[0].set_xlabel('Number of Clusters (Value of K)')
ax[0].set_ylabel('Inertia')

sns.lineplot( x = list(range(2,20)), y = silhouette_list, ax=ax[1])
ax[1].set_xlabel('Number of Clusters (Value of K)')
ax[1].set_ylabel('Silhouette Score')

fig.tight_layout()

In [ ]:
# Chosing the optimal number of clusters from the above plots
choice_cluster_num = 6

In [ ]:
# Obtaining a trained classfication model.
model = train_cluster_classification_model( data = df_merged_filled.loc[train_start:train_end], number_of_clusters = choice_cluster_num)

In [ ]:
# Printing the States found from the classification model for each date in the Dataset
for date_item in dates_in_dataframe[7:]:
  date = f'{date_item.year}-{date_item.month}-{date_item.day}'
  test_df = df_merged_filled.loc[date]
  prediction_labels = model.predict(test_df.drop(['ActivePT'], axis=1))
  print(f'{date} - predicted_labels {set(prediction_labels)}')
  state_details[Location]['states_found'][str(date)] = sorted(set(prediction_labels))

In [ ]:
# Plotting (Scatter Plot) the states found using the classification model
# with X-axis = datetime; and Y-axis = Active Power. 
# Hue argument of the seaborn library is used to represent different states found with different colors.
for date_item in dates_in_dataframe[7:]:
  date = f'{date_item.year}-{date_item.month}-{date_item.day}'
  test_df = df_merged_filled.loc[date]
  prediction_labels = model.predict(test_df.drop(['ActivePT'], axis=1))

  f1_score, _ = get_f1_score(data = test_df.drop(['ActivePT'], axis=1), model = model, num_clusters = choice_cluster_num) 
  f1_score_dic[Location][date] = f1_score

  fig, ax = plt.subplots(2,1, figsize=(12,5))

  principalDf = pca_function(test_df, 2)

  sns.scatterplot(data=test_df, x=test_df.index, y='ActivePT', hue=prediction_labels, palette = palette, ax=ax[0])
  sns.scatterplot( data = principalDf, x="components_1", y="components_2", hue=prediction_labels, palette=palette,ax = ax[1])

  ax[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0)
  ax[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0)

  ax[0].set_title('(a) Cluster with Active Power on Y-Axis').set_fontsize(15)
  ax[0].set_xlabel('datetime')
  ax[1].set_title('(b) PCA ').set_fontsize(15)

  fig.tight_layout()
  # fig.savefig(f'{Location}_{date}')